# P9 : Preprocessing de données visuelles et PCA

Objectif du notebook :

Réaliser en local le processing des données et la réduction de dimensionnalité des vecteurs prédits avec le modèle entrainé dans un code Pyspark en vue d'un futu déploiement dans le cloud AWS

## 1- Import des librairies

In [ ]:
import pandas as pd
from PIL import Image
import numpy as np
import io
import os

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras import Model
from pyspark.sql.functions import col, pandas_udf, PandasUDFType, element_at, split
from pyspark.sql import SparkSession


## 2- Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
PATH = os.getcwd()
PATH_Data = '/content/drive/MyDrive/fruits/fruits-360_dataset/fruits-360/Test'
PATH_Result = PATH+'/data/Results'

In [ ]:
print('PATH:        '+\
      PATH+'\nPATH_Data:   '+\
      PATH_Data+'\nPATH_Result: '+PATH_Result)

PATH:        /content
PATH_Data:   /content/drive/MyDrive/fruits/fruits-360_dataset/fruits-360/Test
PATH_Result: /content/data/Results


## 3- Instanciation des outils Spark

In [ ]:
# création d'une session spark = pt d'entrée dans les fonctionnalités spark
# nécessaire pour contruire un dataframe spark
spark = (SparkSession
             .builder
             .appName('P8')
             .master('local')
             .config("spark.sql.parquet.writeLegacyFormat", 'true')
             .getOrCreate()
)

In [ ]:
# instance d'un sparkcontext
# permet d'accéder aux fonctionnalités de bas niveau de spark
sc = spark.sparkContext

In [ ]:
spark

In [ ]:
# Configurations pour optimiser les performances et la gestion de la mémoire
spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", 1024)
spark.conf.set("spark.sql.shuffle.partitions", 4)

## 4- Loading des images et création d'un dataframe Spark

In [ ]:
# lecture distribuée des images
images = spark.read.format("binaryFile") \
  .option("pathGlobFilter", "*.jpg") \
  .option("recursiveFileLookup", "true") \
  .load(PATH_Data) \
  .limit(2)

In [ ]:
type(images)

pyspark.sql.classic.dataframe.DataFrame

On a un spark dataframe

In [ ]:
# extraction du label depuis le path
images = images.withColumn('label', element_at(split(images['path'], '/'),-2))

In [ ]:
print(images.printSchema())

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)
 |-- label: string (nullable = true)

None


In [ ]:
print(images.select('path','label').show(5,False))

+----------------------------------------------------------------------------------------------+----------+
|path                                                                                          |label     |
+----------------------------------------------------------------------------------------------+----------+
|file:/content/drive/MyDrive/fruits/fruits-360_dataset/fruits-360/Test/Watermelon/r_106_100.jpg|Watermelon|
|file:/content/drive/MyDrive/fruits/fruits-360_dataset/fruits-360/Test/Watermelon/r_109_100.jpg|Watermelon|
+----------------------------------------------------------------------------------------------+----------+

None


## 5- Loading du modèle pré-entrainé et transfert learning

In [ ]:
# loading du modèle pré-entrainé
model = MobileNetV2(weights='imagenet',
                    include_top=True,
                    input_shape=(224, 224, 3))

14536120/14536120 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# suppression de la dernière couche de classification
new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)

In [ ]:
new_model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,257,984 (8.61 MB)

 Trainable params: 2,223,872 (8.48 MB)

 Non-trainable params: 34,112 (133.25 KB)

On obtient en sortie des vecteurs denses de taille 1280

In [ ]:
# broadcast des poids du modèle aux workers spark : permet d'éviter de renvoyer les poids à chaque appel
brodcast_weights = sc.broadcast(new_model.get_weights())

## 5- Inférence sur le modèle et batching des prédictions

In [ ]:
def model_fn():
    """
    Returns a MobileNetV2 model with top layer removed
    and broadcasted pretrained weights.
    """
    model = MobileNetV2(weights='imagenet',
                        include_top=True,
                        input_shape=(224, 224, 3))
    for layer in model.layers:
        layer.trainable = False
    new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)
    new_model.set_weights(brodcast_weights.value)
    return new_model

In [ ]:
def preprocess(content):
    """
    Preprocesses raw image bytes for prediction.
    :return: a preprocessed image tensor
    """
    img = Image.open(io.BytesIO(content)).resize([224, 224])
    arr = img_to_array(img)
    return preprocess_input(arr)

def featurize_series(model, content_series):
    """
    Featurize a pd.Series of raw images using the input model.
    :return: a pd.Series of image features
    """
    input = np.stack(content_series.map(preprocess))
    preds = model.predict(input)
    # For some layers, output features will be multi-dimensional tensors.
    # We flatten the feature tensors to vectors for easier storage in Spark DataFrames.
    output = [p.flatten() for p in preds]
    return pd.Series(output)


# définition d'un pandas UDF scalaire itératif (le format pandas spark)

@pandas_udf('array<float>', PandasUDFType.SCALAR_ITER) # active pandas + Arrow // batch + iterateur
# distribue sur les workers, chaque elt de sortie est un batch de données
def featurize_udf(content_series_iter): # batch spark
    '''
    This method is a Scalar Iterator pandas UDF wrapping our featurization function.
    The decorator specifies that this returns a Spark DataFrame column of type ArrayType(FloatType).

    :param content_series_iter: This argument is an iterator over batches of data, where each batch
                              is a pandas Series of image data.
    '''
    # With Scalar Iterator pandas UDFs, we can load the model once and then re-use it
    # for multiple data batches.  This amortizes the overhead of loading big models.
    model = model_fn()
    for content_series in content_series_iter:
        yield featurize_series(model, content_series) # charge le modèle une fois et renvoie un batch

/usr/local/lib/python3.12/dist-packages/pyspark/sql/pandas/functions.py:446: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


- Spark découpe les données en partitions
- chaque partition est envoyée au worker
- les données sont traitées par batch

content_series = un batch d’images

le modèle est chargé une seule fois par worker

chaque yield renvoie un batch de résultats

les prédictions sont batchées

Spark ne veut pas de float arr mais des vecteurs UDT

=> il faut convertir en vecteur DenseVector

## 6- PCA

Pour faire une PCA avec Spark il faut :

- modifier les vecteurs en DenseVector
- centrer /  réduire les données = standardisation
- importer la PCA Spark
- créer le pipeline

In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import udf
from pyspark.ml.linalg import VectorUDT, Vectors
# modification des vecteurs
array_to_vector = udf(lambda x: Vectors.dense(x), VectorUDT())

1- On va créer une colonne features_array pour associer les arrays à chaque image correspondantes

In [ ]:
df = images.withColumn(
    "features_array",
    featurize_udf("content")
)

In [ ]:
df.printSchema()

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)
 |-- label: string (nullable = true)
 |-- features_array: array (nullable = true)
 |    |-- element: float (containsNull = true)



In [ ]:
df.show()

+--------------------+-------------------+------+--------------------+----------+--------------------+
|                path|   modificationTime|length|             content|     label|      features_array|
+--------------------+-------------------+------+--------------------+----------+--------------------+
|file:/content/dri...|2026-01-06 08:24:00|  7353|[FF D8 FF E0 00 1...|Watermelon|[0.0, 0.23397025,...|
|file:/content/dri...|2026-01-06 08:24:00|  7350|[FF D8 FF E0 00 1...|Watermelon|[0.08841883, 0.83...|
+--------------------+-------------------+------+--------------------+----------+--------------------+



2- On convertit les arrays en vecteurs

In [ ]:
df = df.withColumn(
    "features",
    array_to_vector("features_array")
)

In [ ]:
df.printSchema()

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)
 |-- label: string (nullable = true)
 |-- features_array: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- features: vector (nullable = true)



In [ ]:
df.columns

['path',
 'modificationTime',
 'length',
 'content',
 'label',
 'features_array',
 'features']

On retrouve bien les deux éléments nécessaires à la PCA

1- les arrays<float> du pandas UDF

2- VectorUDT utilisable par Spark MLlib

In [ ]:
# mise en cache du dataframe en mémoire
# permet d'éviter de recalculer les features à chaque action
df = df.cache()
df.count()

2

### 3-  standardisation des données

MobileNetV2 produit des features avec des valeurs sur des échelles différentes
La PCA est très sensible à l’échelle : sinon certaines dimensions vont dominer artificiellement.
On utilise StandardScaler pour ramener à des échelles comparatives

In [ ]:
from pyspark.ml.feature import StandardScaler

In [ ]:
scaler = StandardScaler(
    inputCol="features",
    outputCol="features_scaled",
    withMean=False,   # centre les données (moyenne 0)
    withStd=True     # met à l'échelle avec écart-type 1
)


In [ ]:
scaler_model = scaler.fit(df).transform(df)

In [ ]:
df.columns

['path',
 'modificationTime',
 'length',
 'content',
 'label',
 'features_array',
 'features']

In [ ]:
from pyspark.ml.feature import PCA

In [ ]:
pca = PCA(
    k=5,
    inputCol="features_scaled",
    outputCol="features_pca"
)

In [ ]:
pca_model = pca.fit(scaler_model)
df_pca = pca_model.transform(scaler_model)

In [ ]:
df_pca.select("features_pca").show(truncate=False)
print(df_pca.select("features_pca").first()[0].size)

+-----------------------------------------------------------------------------------------------+
|features_pca                                                                                   |
+-----------------------------------------------------------------------------------------------+
|[144.19234619106516,50.367571810036296,-34.98492965611851,-9.49177457709768,-69.59508653945116]|
|[99.69515526849125,50.36757181003622,-34.98492965611856,-9.491774577097686,-69.59508653945102] |
+-----------------------------------------------------------------------------------------------+

5


In [ ]:
# on check la taille du vecteur
df_pca.select("features_pca").head()[0].size

5

In [ ]:
df_pca.printSchema()

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)
 |-- label: string (nullable = true)
 |-- features_array: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- features: vector (nullable = true)
 |-- features_scaled: vector (nullable = true)
 |-- features_pca: vector (nullable = true)



In [ ]:
print(PATH_Result)

/content/data/Results


In [ ]:
# sauvegarde des résultats
# sauvegarde du dataframe PCA dans un format parquet (format colonne optimisé pour spark)
df_pca.write.mode("overwrite").parquet(PATH_Result)

In [ ]:
# lecture des résultats
df = pd.read_parquet(PATH_Result, engine='pyarrow')